In [ ]:
import pandas as pd

data = pd.read_csv("data/20240114_015.csv", sep=";", encoding="iso-8859-1")
data.head()

In [ ]:
# Drop empty columns
data = data.dropna(axis="columns", how="all")
data.head()

In [ ]:
# Drop columns that have all the same value
same_cols = []
metadata = {}

for colname in data.columns:
    values = data[colname].tolist()
    if len(set(values)) == 1:
        # print(f"Column {colname} is unique with value {col[0]}")
        same_cols.append(colname)
        metadata[colname] = values[0]

metadata

In [ ]:
data = data.drop(columns=same_cols)
data.head()

In [ ]:
# add proper datetime column
from datetime import datetime

data["DATUMTIJD"] = pd.to_datetime(data["WAARNEMINGDATUM"] + " " + data["WAARNEMINGTIJD (MET/CET)"], format="%d-%m-%Y %H:%M:%S")

In [ ]:
# convert X and Y to proper floats
for c in ["X", "Y"]:
    data[c] = data[c].str.replace(",", ".").astype(float)


In [ ]:
# remove rows where X or Y is 0
data = data[data.X != 0]
data = data[data.Y != 0]

In [ ]:
# create locations gdf
import geopandas as gpd

key_col = "LOCATIE_CODE"
attr_cols = ["MEETPUNT_IDENTIFICATIE", "X", "Y"]

locations = data.groupby(by=key_col)[attr_cols].first()
locations = gpd.GeoDataFrame(locations, geometry=gpd.points_from_xy(locations["X"], locations["Y"], crs="EPSG:25831"))

locations.head()

In [ ]:
# convert CRS and scale to -1, 1
locations = locations.to_crs(epsg=28992)

locations["SCALED_X"] = (locations.geometry.x - 155_000) / (325_000 / 2)
locations["SCALED_Y"] = (locations.geometry.y - 463_000) / (325_000 / 2)
locations = locations.round(5)

locations.head()

In [ ]:
# select columns and export locations
locations[["MEETPUNT_IDENTIFICATIE", "SCALED_X", "SCALED_Y"]].to_csv("output/waterhoogtes_locaties.csv", sep=",", encoding="utf-8")

In [ ]:
# pivot to location per column
meas = data.pivot_table(index="DATUMTIJD", values="NUMERIEKEWAARDE", columns="LOCATIE_CODE")
meas.head()

In [ ]:
# remove RWS max values
meas.replace(999_999_999.0, np.nan, inplace=True)
meas

In [ ]:
# show absolute diffs
meas.diff().abs()

In [ ]:
# remove outliers by looking at mean of absolute diff
abs_diff = meas.diff().abs()
outliers = abs_diff >= 5 * abs_diff.mean()
outliers

In [ ]:
# remove outliers
meas[outliers] = np.nan
meas

In [ ]:
# fill gaps by interpolation
meas.interpolate(axis=0, method="linear", limit_direction="both", inplace=True)
meas.astype(int)

In [ ]:
# back to long table and export
meas_long = meas.astype(int).reset_index().melt(id_vars="DATUMTIJD", value_name="NUMERIEKEWAARDE")
meas_long.to_csv("output/waterhoogtes_long.csv", sep=";", encoding="utf-8", index=False)

In [ ]:
# pivot for touch and export
meas_piv = meas_long.pivot_table(index="LOCATIE_CODE", values="NUMERIEKEWAARDE", columns="DATUMTIJD").astype(int)
meas_piv.to_csv("output/waterhoogtes_pivoted.csv", sep=",", encoding="utf-8")